# Introduction to Generative LLMs for Social Science Research

June Yang, CSDE and eScience Institute, Winter 2024

Hello! Welcome to the workshop on using LLMs for social science research.

Today we'll be using Google's Gemini 1.5 Flash model to annotate text.

Table of Contents

# Resources and Materials

- [OpenAI API Reference](https://platform.openai.com/docs/api-reference)
- [LangChain Documentation](https://docs.langchain.com/docs/)
- [Parlance Labs](https://parlance-labs.com/education/prompt_eng/berryman.html)
- [The landscape of LLMs and their openness](https://opening-up-chatgpt.github.io)
- [eScience SSEC tutorial](https://uw-ssec-tutorials.readthedocs.io/en/latest/SciPy2024/README.html) 
- [Hugging Face](https://huggingface.co)
- Scholars of the field


https://www.youtube.com/watch?v=BrsocJb-fAo

https://www.youtube.com/watch?v=aywZrzNaKjs

https://github.com/rabbitmetrics/langchain-13-min/blob/main/notebooks/langchain-13-min.ipynb

https://github.com/svpino/youtube-rag



# What is a generative LLM?

[BERT vs. GPT: A Tale of Two Transformers that Revolutionized NLP](https://medium.com/@prudhvithtavva/bert-vs-gpt-a-tale-of-two-transformers-that-revolutionized-nlp-11fff8e61984)

Generative vs. Discriminative Models

	•	Generative Models: These models learn to generate new data instances that resemble the training data. In language models, this means they can produce text, complete sentences, and engage in open-ended tasks like translation or summarization. Examples include GPT (Generative Pre-trained Transformer) and other models built on transformer architectures.
	
	•	Discriminative (or Classifier) Models: These models focus on making predictions or classifications based on input data. For example, they might classify sentiment, identify named entities, or categorize topics. Models like BERT (Bidirectional Encoder Representations from Transformers) are discriminative since they’re optimized for predicting specific labels given input text, such as in sentiment analysis or question answering.

# The use of generative LLMs for the social sciences

Reference: Large Language Models (LLMs) in Social Science Research Session I: Introduction

Joshua Cova and Luuk Schmitz, MPISS, 20-06-2024

1. Modelling human behavior computationally
   
   1.1. Testing & (potentially) running experiments: Aher, Arriaga, and Kalai (2022); Dillion et al(2023)

   1.2. Running surveys: Tjuatia et al.(2023)

2. Simulating social relationships
   
   2.1. Simulating social networks: Interactions between articifial agents: Park et al.(2023); Wang et al.(2024)

   2.2. Game theoretical simulations: Akata et al.(2023)

3. Interacting with human agents
   
   3.1. Chatbots for interviewing partiticipants: Chopra and Haaland (2023)

4. Text annotation
   
   4.1. Zero-shot & few-shot text annotation: Törnberg (2023); Gilardi, Alizadeh, and Kubi (2023); Leek, Bischl, and Freier (2024)
   4.2. Synthetic data generation: Laurer (2024)


LLMs can also help streamline the research pipeline more generally. 

- Spitballing ideas
- Data preparation
- Summazing texts (e.g., Interview data)
- Creative writing
- Conducting preliminary analyses 
  
See also Korinek (2023)

# How to use generative LLMs beyond chat?


Introducing the `langchain` package.

In this tutorial, we're going to replicate Törnberg (2023): [How to use Large Language Models for Text Analysis](https://arxiv.org/abs/2307.13106) by breaking the task into process using the `langchain` package. 

What is the paper doing?

Before we start, there are some important concepts to understand.

    - What is an API?
    - Check out the OpenAI API and playground

## Using Chat Completion for Text Annotation

### Step 1. Models (LLM Wrappers)

In [1]:
# install packages in your environment:
# pip install langchain openai pandas scikit-learn dotenv

from dotenv import load_dotenv,find_dotenv
from langchain_community.llms import OpenAI
import pandas as pd
import os
import chardet
import time  

# optional: suppress warnings
import warnings
warnings.filterwarnings('ignore')
# Specifically for LangChain deprecation warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)

In [2]:
# Set up OpenAI API key
load_dotenv(find_dotenv())

# Initialize the LLM
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0.2)

# Test the LLM
print(llm.invoke("Hello, world!"))

Hello! How can I assist you today?


Dataset: Hawkins, Kirk A., Rosario Aguilar, Erin Jenne, Bojana Kocijan, Cristóbal Rovira Kaltwasser, Bruno Castanho Silva. 2019. Global Populism Database: Populism Dataset for Leaders 1.0. [link](https://populism.byu.edu/data/2019%20-%20global%20populism%20database%20(guardian%20version))


Auto encoding:

In [3]:
def read_file_with_auto_encoding(file_path):
    # First, detect the encoding
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            text = file.read()
            # Remove BOM if present
            if text.startswith('\ufeff'):
                text = text[1:]
                
            # Find the actual speech content
            lines = text.split('\n')
            content_start = 0
            
            # First try: Look for quotation marks
            for i, line in enumerate(lines):
                if '"' in line or '"' in line:
                    content_start = i
                    break
            
            # Second try: Look for first substantial text after metadata
            if content_start == 0:
                empty_lines = 0
                for i, line in enumerate(lines):
                    if not line.strip():
                        empty_lines += 1
                    elif empty_lines >= 2:  # After finding 2+ empty lines, next non-empty is content
                        content_start = i
                        break
            
            # Third try: Look for first empty line (simpler fallback)
            if content_start == 0:
                for i, line in enumerate(lines):
                    if line.strip() == '' and i > 0:
                        content_start = i + 1
                        break
            
            # Get the content
            content = '\n'.join(lines[content_start:])
            
            # Clean up any leading/trailing whitespace
            content = content.strip()
            
            return content
            
    except UnicodeDecodeError:
        # Fallback to latin-1 if detection fails
        with open(file_path, 'r', encoding='latin-1') as file:
            text = file.read()
            if text.startswith('\ufeff'):
                text = text[1:]
            return text.strip()
    except Exception as e:
        print(f"Error reading file {file_path}: {str(e)}")
        return ""

In [4]:
# Path to the folder containing speech text files
folder_path = 'data/global-populism-dataset-zi/speeches_20220427/speeches_20220427'
# Initialize an empty list to store speech data
data = []

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        try:
            text = read_file_with_auto_encoding(file_path)
            speech_name = os.path.splitext(filename)[0]
            data.append({'speech_name': speech_name, 'text': text})
        except Exception as e:
            print(f"Failed to read file {filename}: {str(e)}")
            continue

# Create a DataFrame with the collected data
df = pd.DataFrame(data)
print("\nFirst few rows:")
print(df.head())

# Print a sample of text from each row to verify encoding
print("\nSample of first 100 characters from each text:")
for idx, row in df.head().iterrows():
    print(f"\n{row['speech_name']}:")
    print(row['text'][:100])


First few rows:
                              speech_name  \
0               Nicaragua_Ortega_Famous_1   
1                  France_Chirac_Famous_1   
2                   Serbia_Tadic_Famous_1   
3  Georgia_Margvelashvili_International_1   
4                       UK_Blair_Ribbon_3   

                                                text  
0  Primer Año del Gobierno del Pueblo\nPrimer Ani...  
1  "Déclaration aux Français de Monsieur Jacques ...  
2  Држава Србија је забринута што је изостала реа...  
3                                                     
4  A Cabinet Minister is sitting having a pub lun...  

Sample of first 100 characters from each text:

Nicaragua_Ortega_Famous_1:
Primer Año del Gobierno del Pueblo
Primer Aniversario del Gobierno de 
Reconciliación y Unidad Nacio

France_Chirac_Famous_1:
"Déclaration aux Français de Monsieur Jacques CHIRAC, Président de la République.

Palais de l'Elysé

Serbia_Tadic_Famous_1:
Држава Србија је забринута што је изостала реакција Ује

Chunk text

In [5]:
import tiktoken
import nltk
from nltk.tokenize import sent_tokenize

# Download NLTK sentence tokenizer
# nltk.download('punkt')

# Initialize the tokenizer for GPT-3.5-turbo
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Define the maximum token limit for GPT-3.5-turbo
max_tokens = 4096

# Function to count the number of tokens in a text
def count_tokens(text):
    return len(encoding.encode(text))

# Function to split text into chunks based on token limits
def split_text_into_chunks(text, max_tokens=max_tokens):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_length = 0
    
    for sentence in sentences:
        # Skip empty sentences
        if not sentence.strip():
            continue
            
        sentence_length = count_tokens(sentence)
        
        # Handle very long sentences
        if sentence_length > max_tokens:
            if current_chunk:
                chunks.append(' '.join(current_chunk))
                current_chunk = []
                current_length = 0
            # Split long sentence into smaller pieces
            words = sentence.split()
            temp_chunk = []
            temp_length = 0
            for word in words:
                word_length = count_tokens(word)
                if temp_length + word_length < max_tokens:
                    temp_chunk.append(word)
                    temp_length += word_length
                else:
                    chunks.append(' '.join(temp_chunk))
                    temp_chunk = [word]
                    temp_length = word_length
            if temp_chunk:
                chunks.append(' '.join(temp_chunk))
            continue
            
        if current_length + sentence_length <= max_tokens:
            current_chunk.append(sentence)
            current_length += sentence_length
        else:
            chunks.append(' '.join(current_chunk))
            current_chunk = [sentence]
            current_length = sentence_length

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

# Apply token check and chunking to each text in the DataFrame
df['chunks'] = df['text'].apply(lambda x: split_text_into_chunks(x))

# Print the first few rows to verify
print(df[['speech_name', 'chunks']].head())

                              speech_name  \
0               Nicaragua_Ortega_Famous_1   
1                  France_Chirac_Famous_1   
2                   Serbia_Tadic_Famous_1   
3  Georgia_Margvelashvili_International_1   
4                       UK_Blair_Ribbon_3   

                                              chunks  
0  [Primer Año del Gobierno del Pueblo\nPrimer An...  
1  ["Déclaration aux Français de Monsieur Jacques...  
2  [Држава Србија је забринута што је изостала ре...  
3                                                 []  
4  [A Cabinet Minister is sitting having a pub lu...  


In [9]:
# Get length of chunks list for each row
df['chunk_count'] = df['chunks'].apply(len)

# Show distribution of chunk counts
print("Chunk count distribution:")
print(df['chunk_count'].value_counts().sort_index())

# Show some example rows with their chunk counts
print("\nExample rows with chunk counts:")
print(df[['speech_name', 'chunk_count']].head())

# Get summary statistics
print("\nSummary statistics of chunk counts:")
print(df['chunk_count'].describe())

Chunk count distribution:
chunk_count
0     111
1     645
2     258
3      73
4      34
5      14
6       9
7       5
8       4
9       2
10      2
11      3
14      1
Name: count, dtype: int64

Example rows with chunk counts:
                              speech_name  chunk_count
0               Nicaragua_Ortega_Famous_1            4
1                  France_Chirac_Famous_1            1
2                   Serbia_Tadic_Famous_1            2
3  Georgia_Margvelashvili_International_1            0
4                       UK_Blair_Ribbon_3            2

Summary statistics of chunk counts:
count    1161.000000
mean        1.543497
std         1.379672
min         0.000000
25%         1.000000
50%         1.000000
75%         2.000000
max        14.000000
Name: chunk_count, dtype: float64


Remove the zero-chunk rows.

In [9]:
# Remove rows where chunks list is empty
df_clean = df[df['chunks'].apply(len) > 0].copy()

# Print before/after stats
print(f"Original number of speeches: {len(df)}")
print(f"Number after removing empty chunks: {len(df_clean)}")
print(f"Removed {len(df) - len(df_clean)} speeches")

# Reset the index if needed
df_clean = df_clean.reset_index(drop=True)

Original number of speeches: 1161
Number after removing empty chunks: 1050
Removed 111 speeches


In [11]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   speech_name  1050 non-null   object
 1   text         1050 non-null   object
 2   chunks       1050 non-null   object
dtypes: object(3)
memory usage: 24.7+ KB


### Step 2. Prompts

prompt engineering

In [12]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage, HumanMessage
# Define the base prompt template
system_prompt = """You are an expert in analyzing political speeches for populist content. 
You can analyze text in any language."""

human_prompt = """Your task is to evaluate the level of populism in this political text:

{text}

A populist text must contain BOTH of these elements:

1. People-centrism:
- Focus on "the people" or "ordinary people" as an indivisible/homogeneous community
- Promotes politics as the popular will of "the people"
- NOTE: Appeals to specific subgroups (ethnicities, regional groups, classes) are NOT populist

2. Anti-elitism:
- Focus on "the elite" with negative descriptions
- Presents elite vs people as a moral struggle between good and bad
- NOTE: Criticism of specific elite members is NOT populist - must reject elite as a whole

Rate from 0-2:
0 = Not populist
1 = Somewhat populist
2 = Highly populist

Respond with: [score]; [brief justification]"""

# Create the chat prompt template
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", human_prompt),
])

### Step 3. Chains

Let's first test the prompt with a single chunk

In [14]:
from langchain import LLMChain
# Create the chain
chain = LLMChain(llm=llm, prompt=prompt_template)

# Test prompt with single chunk
try:
    # Use the chain we created earlier
    response = chain.run(text=test_chunk)
    
    print("Input chunk preview:")
    print(test_chunk[:300], "...\n")
    
    print("LLM Response:")
    score, justification = response.split(';', 1)
    print(f"Score: {score.strip()}")
    print(f"Justification: {justification.strip()}")
    
except Exception as e:
    print(f"Error processing text: {str(e)}")

/var/folders/zj/d9h6gl49125919pfb15tp_gc0000gn/T/ipykernel_8878/1671775501.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)
/var/folders/zj/d9h6gl49125919pfb15tp_gc0000gn/T/ipykernel_8878/1671775501.py:8: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = chain.run(text=test_chunk)


Input chunk preview:
Primer Año del Gobierno del Pueblo
Primer Aniversario del Gobierno de 
Reconciliación y Unidad Nacional, 
Gobierno del Poder Ciudadano 
Plaza de la Revolución

10 de enero del 2007 


“Quiero, ante ustedes, citar nuevamente al Papa Benedicto XVI. Juan Pablo decía: capitalismo salvaje, los pueblos no ...

LLM Response:
Score: 2
Justification: The text contains a strong emphasis on "the people" and their power, as well as a clear anti-elitist sentiment, criticizing those who seek to raise prices on basic goods and highlighting the importance of protecting the rights of the people. The speech also references the support of various leaders and countries that are seen as allies of the people, further emphasizing a populist narrative.


In [16]:
from langchain import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template)

## Process all rows in the dataframe
results = []

for index, row in df_clean.iterrows():
    print(f"\nProcessing speech {index + 1}/{len(df_clean)}: {row['speech_name']}")
    
    # Process each chunk in the speech
    chunk_scores = []
    chunk_justifications = []
    
    for chunk in row['chunks']:
        try:
            response = chain.run(text=chunk)  # Process raw chunk directly
            score, justification = response.split(';', 1)
            chunk_scores.append(float(score.strip()))
            chunk_justifications.append(justification.strip())
            
            time.sleep(1)
            
        except Exception as e:
            print(f"Error processing chunk: {str(e)}")
            continue
    
    # Calculate average score and combine justifications
    if chunk_scores:
        avg_score = sum(chunk_scores) / len(chunk_scores)
        results.append({
            'speech_name': row['speech_name'],
            'populism_score': round(avg_score, 2),
            'num_chunks_processed': len(chunk_scores),
            'justifications': chunk_justifications
        })


Processing speech 1/1050: Nicaragua_Ortega_Famous_1

Processing speech 2/1050: France_Chirac_Famous_1

Processing speech 3/1050: Serbia_Tadic_Famous_1

Processing speech 4/1050: UK_Blair_Ribbon_3

Processing speech 5/1050: Malaysia_Mohamad_International_1

Processing speech 6/1050: India_Singh_International_2

Processing speech 7/1050: Honduras_Lobo Sosa_Famous_1

Processing speech 8/1050: Netherlands_Balkenende_Famous_2

Processing speech 9/1050: South Africa_Ramaphosa_International_1

Processing speech 10/1050: Russia_Putin_Famous_2

Processing speech 11/1050: Romania_Ponta_Campaign_1

Processing speech 12/1050: Canada_Harper_Famous_1

Processing speech 13/1050: Argentina_Fernandez_Famous_1

Processing speech 14/1050: Lithuania_Kubilius_Famous_1

Processing speech 15/1050: Ecuador_Correa_Famous_2

Processing speech 16/1050: Macedonia_Gruevski_Ribbon_2

Processing speech 17/1050: Singapore_Loong_Ribbon_1

Processing speech 18/1050: Mexico_Pena Nieto_Ribbon_1

Processing speech 19/105

In [17]:
# Create final results dataframe
results_df = pd.DataFrame(results)

# Display summary
print("\nAnalysis complete!")
print(f"Processed {len(results_df)} speeches")
print("\nFirst few results:")
print(results_df.head())


Analysis complete!
Processed 1045 speeches

First few results:
                        speech_name  populism_score  num_chunks_processed  \
0         Nicaragua_Ortega_Famous_1             2.0                     4   
1            France_Chirac_Famous_1             2.0                     1   
2             Serbia_Tadic_Famous_1             2.0                     2   
3                 UK_Blair_Ribbon_3             1.0                     2   
4  Malaysia_Mohamad_International_1             2.0                     1   

                                      justifications  
0  [This political text contains strong elements ...  
1  [This political text contains strong elements ...  
2  [The text contains strong elements of people-c...  
3  [The text contains elements of people-centrism...  
4  [The speech contains strong elements of populi...  


In [18]:
# Save to CSV
results_df.to_csv('data/populism_analysis_results2.csv', index=False)


### Step 4. Validation

In [20]:
import numpy as np
from krippendorff import alpha

# First, create a mapping of speech names to their indices in the original df
original_speech_map = {name: idx for idx, name in enumerate(df['speech_name'])}

# Add random ground truth scores (0, 1, or 2) for the original dataframe
np.random.seed(42)  # for reproducibility
df['ground_truth'] = np.random.choice([0, 1, 2], size=len(df))

# Create a merged dataframe that includes both LLM scores and ground truth
merged_df = pd.merge(
    results_df,
    df[['speech_name', 'ground_truth']],
    on='speech_name',
    how='inner'
)

print("Number of speeches in original df:", len(df_clean))
print("Number of speeches in results_df:", len(results_df))
print("Number of matched speeches:", len(merged_df))


Number of speeches in original df: 1050
Number of speeches in results_df: 1045
Number of matched speeches: 1045


In [22]:
# Find speeches that weren't processed by the model
unprocessed_df = df[~df['speech_name'].isin(results_df['speech_name'])]

print("Analysis of missing annotations:")
print(f"Total speeches in original dataset: {len(df)}")
print(f"Speeches processed by model: {len(results_df)}")
print(f"Speeches missing annotations: {len(unprocessed_df)}")

# Display some details about the unprocessed speeches
if len(unprocessed_df) > 0:
    print("\nSample of unprocessed speeches:")
    print(unprocessed_df[['speech_name', 'chunks']].head())

Analysis of missing annotations:
Total speeches in original dataset: 1161
Speeches processed by model: 1045
Speeches missing annotations: 116

Sample of unprocessed speeches:
                               speech_name chunks
3   Georgia_Margvelashvili_International_1     []
15          Kazakhstan_Nazarbayev_Ribbon_2     []
24           Estonia_Ansip_International_3     []
27           Estonia_Ansip_International_2     []
37        Bulgaria_Borisov_International_1     []


In [23]:
# Examine the original text of speeches with empty chunks
print("Sample of speeches with empty chunks:")
for idx, row in unprocessed_df.head().iterrows():
    speech_name = row['speech_name']
    original_text = df[df['speech_name'] == speech_name]['text'].values[0]
    print(f"\n{speech_name}:")
    print(f"First 200 characters: {original_text[:200]}")
    print(f"Text length: {len(original_text)}")
    print(f"Is text empty? {len(original_text.strip()) == 0}")

Sample of speeches with empty chunks:

Georgia_Margvelashvili_International_1:
First 200 characters: 
Text length: 0
Is text empty? True

Kazakhstan_Nazarbayev_Ribbon_2:
First 200 characters: 
Text length: 0
Is text empty? True

Estonia_Ansip_International_3:
First 200 characters: 
Text length: 0
Is text empty? True

Estonia_Ansip_International_2:
First 200 characters: 
Text length: 0
Is text empty? True

Bulgaria_Borisov_International_1:
First 200 characters: 
Text length: 0
Is text empty? True


Illustration purpose only.

In [24]:
# Prepare data for Krippendorff's alpha
# Convert to matrix where each row is a speech and each column is a rater
reliability_data = np.array([
    merged_df['populism_score'].values,
    merged_df['ground_truth'].values
])

# Calculate Krippendorff's alpha
k_alpha = alpha(reliability_data=reliability_data.T, level_of_measurement='ordinal')

print("\nKrippendorff's alpha:", k_alpha)

# Basic agreement statistics
agreement_df = merged_df.copy()
agreement_df['exact_match'] = agreement_df['populism_score'] == agreement_df['ground_truth']
agreement_df['diff'] = abs(agreement_df['populism_score'] - agreement_df['ground_truth'])

print("\nAgreement Statistics:")
print(f"Exact matches: {agreement_df['exact_match'].mean():.2%}")
print(f"Average difference: {agreement_df['diff'].mean():.2f}")

# Distribution of scores
print("\nScore Distribution:")
print("LLM Scores:")
print(merged_df['populism_score'].value_counts().sort_index())
print("\nGround Truth:")
print(merged_df['ground_truth'].value_counts().sort_index())

# Save the merged results
merged_df.to_csv('data/populism_analysis_with_ground_truth2.csv', index=False)


Krippendorff's alpha: 0.09299251372378137

Agreement Statistics:
Exact matches: 27.08%
Average difference: 0.87

Score Distribution:
LLM Scores:
populism_score
0.00     43
0.33      2
0.50     14
0.57      1
0.62      1
0.67      4
0.75      1
0.80      2
1.00    359
1.12      1
1.20      1
1.25      2
1.33     17
1.36      1
1.40      1
1.43      2
1.44      1
1.45      1
1.50     79
1.57      1
1.60      4
1.62      2
1.67     17
1.73      1
1.75      9
1.80      1
1.86      1
2.00    476
Name: count, dtype: int64

Ground Truth:
ground_truth
0    370
1    338
2    337
Name: count, dtype: int64


## Advanced Topic. Embeddings and VectorStores

Understanding word embedding

What is RAG?

Differences between RAG and Fine-Tuning

# Model Selection

# Challenges